# Deploy comunication compression scheme in FedLab

This tutorial provides comprehensive examples about implementing a communication efficiency scheme in FedLab. 

We take the baseline gradient compression algorithms as examples (top-k for gradient sparsification and QSGD for gradient quantization).

## Compress example

In [1]:
import sys
sys.path.append("../")

from fedlab.contrib.compressor.quantization import QSGDCompressor
from fedlab.contrib.compressor.topk import TopkCompressor
import torch

tpk_compressor = TopkCompressor(compress_ratio=0.05) # top 5% gradient
qsgd_compressor = QSGDCompressor(n_bit=8)


In [2]:
# top-k
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
values, indices = tpk_compressor.compress(tensor)
print("Compressed results top-k values:",values)
print("Compressed results top-k indices:", indices)

# decompress
decompressed = tpk_compressor.decompress(values, indices, shape)
print("Decompressed results:", decompressed)

To be compressed tensor: tensor([ 1.7904,  1.1976,  0.5574, -1.4732, -0.1749, -1.5302,  0.7105,  0.8731,
        -0.2417, -1.7519, -0.8683,  1.6638, -0.8791,  0.3522,  1.2140, -0.7861,
        -0.6471,  0.1058,  0.9712, -0.0356,  0.1277,  0.6797,  0.7012, -0.0314,
         0.2698,  0.4957,  1.5576, -0.4206, -1.5119, -0.2276, -0.1169,  0.0241,
         1.1393,  1.1940,  0.1736, -0.8024,  0.5382, -0.0825,  0.7265, -0.0350,
        -0.9015,  0.9518,  0.1257, -1.3379,  1.2557, -0.1562,  0.2343, -0.2276,
         0.1734, -0.7848,  0.4302, -1.5303, -1.2221,  0.8777,  0.7470,  0.8461,
        -0.8365,  0.1586, -0.0298, -0.9709, -0.2771, -0.4565,  0.5078,  1.1161,
        -2.9212,  0.7157,  1.0265, -0.3000, -1.1337, -0.7994,  1.4019,  0.5700,
         1.0981,  1.7451, -0.6322, -0.5967,  1.3718, -1.2059, -0.6930,  0.3289,
        -0.2357, -0.2602,  0.4087,  2.4009,  0.4228, -0.1829,  1.1983,  1.9031,
         1.2283,  1.1762, -0.7700,  0.7999, -0.7868, -2.4056,  0.3990,  0.3036,
         0.7046

In [3]:
# qsgd
tensor = torch.randn(size=(100,))
shape = tensor.shape
print("To be compressed tensor:", tensor)

# compress
norm, signs, values = qsgd_compressor.compress(tensor)
print("Compressed results QSGD norm:", norm)
print("Compressed results QSGD signs:", signs)
print("Compressed results QSGD values:", values)


To be compressed tensor: tensor([-1.8558,  0.1828,  1.1786, -0.1234,  0.5163, -1.2538,  0.3737, -0.1130,
        -0.9324, -0.1437, -0.4652, -0.0538, -0.7610, -1.2260,  0.7752, -0.2309,
         0.0810, -0.2054, -1.2495,  0.0275, -0.0130, -1.4626,  0.0565,  0.1517,
         0.9598, -2.0691, -1.0476,  0.5621, -0.1763, -0.6827, -1.9742, -0.4692,
         0.3654, -0.5896,  1.0840,  1.1551,  0.0322,  0.0544, -0.6744, -0.1711,
        -0.7766,  0.2856, -1.4287,  0.6842,  0.1862,  1.5727, -0.2767,  0.3260,
         2.3558, -0.9370, -0.6398, -1.4415,  0.0954, -0.9119, -0.0301, -0.8488,
        -0.5083, -1.0651, -0.4212,  0.4079, -0.1578, -0.0625, -0.2592, -1.0756,
         1.7337,  0.0145, -0.0273, -0.0423,  0.1000,  2.4316, -0.6455,  0.6802,
        -1.1766, -1.6501,  0.0963, -0.6059,  1.0251,  1.1533, -0.6849,  0.2401,
         0.2348,  0.8670,  2.0629,  0.4358,  1.4603, -0.2489, -0.4500,  1.0784,
         0.6003,  0.1710, -0.7972,  0.1425, -0.3223,  0.7629,  1.6252,  0.0471,
        -0.3190

In [4]:
# decompress
decompressed = qsgd_compressor.decompress([norm, signs, values])
print("Decompressed results:", decompressed)

Decompressed results: tensor([-1.8522,  0.1805,  1.1778, -0.1235,  0.5129, -1.2538,  0.3799, -0.1140,
        -0.9308, -0.1425, -0.4654, -0.0570, -0.7599, -1.2253,  0.7789, -0.2375,
         0.0760, -0.2090, -1.2443,  0.0285, -0.0095, -1.4628,  0.0570,  0.1520,
         0.9593, -2.0706, -1.0448,  0.5604, -0.1805, -0.6839, -1.9757, -0.4749,
         0.3704, -0.5889,  1.0828,  1.1588,  0.0285,  0.0475, -0.6744, -0.1710,
        -0.7694,  0.2850, -1.4248,  0.6839,  0.1805,  1.5767, -0.2755,  0.3229,
         2.3556, -0.9403, -0.6459, -1.4438,  0.0950, -0.9118, -0.0285, -0.8454,
        -0.5129, -1.0638, -0.4179,  0.4084, -0.1615, -0.0570, -0.2565, -1.0828,
         1.7287,  0.0095, -0.0285, -0.0475,  0.1045,  2.4316, -0.6459,  0.6744,
        -1.1778, -1.6527,  0.0950, -0.6079,  1.0258,  1.1588, -0.6839,  0.2470,
         0.2375,  0.8644,  2.0612,  0.4274,  1.4533, -0.2470, -0.4559,  1.0733,
         0.5984,  0.1710, -0.7979,  0.1425, -0.3229,  0.7599,  1.6337,  0.0475,
        -0.3134,  

## Use compressor in federated learning

For example on the client side, we could compress the tensors are to compressed and upload the compressed results to server. And server could decompress the tensors follows the compression agreements.

In jupyter notebook, we take the standalone scenario as example.

In [5]:
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

class CompressSerialClientTrainer(SGDSerialClientTrainer):
    def setup_compressor(self, compressor):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor

    @property
    def uplink_package(self):
        package = super().uplink_package
        new_package = []
        for content in package:
            pack = [self.compressor.compress(content[0])]
            new_package.append(pack)
        return new_package

class CompressServerHandeler(SyncServerHandler):
    def setup_compressor(self, compressor, type):
        #self.compressor = TopkCompressor(compress_ratio=k)
        self.compressor = compressor
        self.type = type

    def load(self, payload) -> bool:
        if self.type == "topk":
            values, indices = payload[0]
            decompressed_payload = self.compressor.decompress(values, indices, self.model_parameters.shape)

        if self.type == "qsgd":
            n, s, l = payload[0]
            decompressed_payload = self.compressor.decompress((n,s,l))
        
        return super().load([decompressed_payload])

In [6]:
# main, this part we follow the pipeline in pipeline_tutorial.ipynb
# But replace the hander and trainer by the above defined for communication compression

# configuration
from opcode import cmp_op
from munch import Munch
from fedlab.models.mlp import MLP

model = MLP(784, 10)
args = Munch

args.total_client = 100
args.alpha = 0.5
args.seed = 42
args.preprocess = False
args.cuda = False
args.cmp_op = "qsgd" # "topk, qsgd"

args.k = 0.1 # topk
args.bit = 8 # qsgd

if args.cmp_op == "topk":
    compressor = TopkCompressor(args.k)

if args.cmp_op == "qsgd":
    compressor = QSGDCompressor(args.bit)

from torchvision import transforms
from fedlab.contrib.dataset.partitioned_mnist import PartitionedMNIST

fed_mnist = PartitionedMNIST(root="../datasets/mnist/",
                             path="../datasets/mnist/fedmnist/",
                             num_clients=args.total_client,
                             partition="noniid-labeldir",
                             dir_alpha=args.alpha,
                             seed=args.seed,
                             preprocess=args.preprocess,
                             download=True,
                             verbose=True,
                             transform=transforms.Compose([
                                 transforms.ToPILImage(),
                                 transforms.ToTensor()
                             ]))

dataset = fed_mnist.get_dataset(0)  # get the 0-th client's dataset
dataloader = fed_mnist.get_dataloader(
    0,
    batch_size=128)  # get the 0-th client's dataset loader with batch size 128


In [7]:
# client
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer, SGDClientTrainer

# local train configuration
args.epochs = 5
args.batch_size = 128
args.lr = 0.1

trainer = CompressSerialClientTrainer(model, args.total_client,
                                 cuda=args.cuda)  # serial trainer
# trainer = SGDClientTrainer(model, cuda=True) # single trainer

trainer.setup_dataset(fed_mnist)
trainer.setup_optim(args.epochs, args.batch_size, args.lr)
trainer.setup_compressor(compressor)

# server
from fedlab.contrib.algorithm.basic_server import SyncServerHandler

# global configuration
args.com_round = 10
args.sample_ratio = 0.1

handler = CompressServerHandeler(model=model,
                            global_round=args.com_round,
                            sample_ratio=args.sample_ratio,
                            cuda=args.cuda)
handler.setup_compressor(compressor, args.cmp_op)

In [8]:
from fedlab.utils.functional import evaluate
from fedlab.core.standalone import StandalonePipeline

from torch import nn
from torch.utils.data import DataLoader
import torchvision

class EvalPipeline(StandalonePipeline):
    def __init__(self, handler, trainer, test_loader):
        super().__init__(handler, trainer)
        self.test_loader = test_loader

    def main(self):
        while self.handler.if_stop is False:
            # server side
            sampled_clients = self.handler.sample_clients()
            broadcast = self.handler.downlink_package

            # client side
            self.trainer.local_process(broadcast, sampled_clients)
            uploads = self.trainer.uplink_package

            # server side
            for pack in uploads:
                self.handler.load(pack)

            loss, acc = evaluate(self.handler.model, nn.CrossEntropyLoss(),
                                 self.test_loader)
            print("loss {:.4f}, test accuracy {:.4f}".format(loss, acc))


test_data = torchvision.datasets.MNIST(root="../datasets/mnist/",
                                       train=False,
                                       transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=1024)

standalone_eval = EvalPipeline(handler=handler,
                               trainer=trainer,
                               test_loader=test_loader)
standalone_eval.main()

[6, 25, 40, 57, 61, 76, 84, 87, 89, 92]
loss 21.9241, test accuracy 0.2840
[2, 12, 17, 21, 39, 47, 49, 53, 65, 94]
